# Question 1

In [1]:
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [2]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url,'lxml')

In [3]:
My_table = soup.find('table',{'class':'wikitable sortable'})
My_table

In [4]:
print(My_table.tr.text)

AttributeError: 'NoneType' object has no attribute 'tr'

In [5]:
headers="Postcode,Borough,Neighbourhood"

In [6]:
table1=""
for tr in My_table.find_all('tr'):
    row1=""
    for tds in tr.find_all('td'):
        row1=row1+","+tds.text
    table1=table1+row1[1:]
print(table1)

AttributeError: 'NoneType' object has no attribute 'find_all'

In [7]:
file=open("toronto.csv","wb")
#file.write(bytes(headers,encoding="ascii",errors="ignore"))
file.write(bytes(table1,encoding="ascii",errors="ignore"))

0

Converting into dataframe and assigning columnnames

In [8]:
import pandas as pd
df = pd.read_csv('toronto.csv',header=None)
df.columns=["Postalcode","Borough","Neighbourhood"]

EmptyDataError: No columns to parse from file

In [9]:
df.head(10)

NameError: name 'df' is not defined

###  Only processing the cells that have an assigned borough. Ignoring the cells with a borough that is Not assigned. Droping row where borough is "Not assigned"

In [10]:
# Get names of indexes for which column Borough has value "Not assigned"
indexNames = df[ df['Borough'] =='Not assigned'].index

# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)

NameError: name 'df' is not defined

In [11]:
df.head(10)

NameError: name 'df' is not defined

### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [12]:
df.loc[df['Neighbourhood'] =='Not assigned' , 'Neighbourhood'] = df['Borough']
df.head(10)

NameError: name 'df' is not defined

### Rows will be same postalcode will combined into one row with the neighborhoods separated with a comma

In [13]:
result = df.groupby(['Postalcode','Borough'], sort=False).agg( ', '.join)

NameError: name 'df' is not defined

In [14]:
df_new=result.reset_index()
df_new.head(15)

NameError: name 'result' is not defined

### use the .shape method to print the number of rows of your dataframe

In [37]:
df_new.shape

(103, 3)

# Question 2

In [15]:
!wget -q -O 'Toronto_long_lat_data.csv'  http://cocl.us/Geospatial_data
df_lon_lat = pd.read_csv('Toronto_long_lat_data.csv')
df_lon_lat.head()

'wget' is not recognized as an internal or external command,
operable program or batch file.


FileNotFoundError: [Errno 2] No such file or directory: 'Toronto_long_lat_data.csv'

In [16]:
df_lon_lat.columns=['Postalcode','Latitude','Longitude']

NameError: name 'df_lon_lat' is not defined

In [17]:
df_lon_lat.head()

NameError: name 'df_lon_lat' is not defined

In [18]:

Toronto_df = pd.merge(df_new,
                 df_lon_lat[['Postalcode','Latitude', 'Longitude']],
                 on='Postalcode')
Toronto_df

NameError: name 'df_new' is not defined

# Question 3
Explore and cluster the neighborhoods in Toronto
Use geopy library to get the latitude and longitude values of New York City.

In [19]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [20]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [46]:
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(Toronto_df['Latitude'], Toronto_df['Longitude'], Toronto_df['Borough'], Toronto_df['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [21]:
CLIENT_ID = 'BTCAEYRO1UYJBXNTS5WIGVWOAPAIMD543VKNEQD10DFDPW05' # your Foursquare ID

CLIENT_SECRET = 'BOYIADY50USFK3SDBSB52P3BAADB1YCET3ELHXIXWULXIQD2' # your Foursquare Secret

VERSION = 'ymd' # Foursquare API version

print('My credentails: ***')

print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails: ***
CLIENT_ID: BTCAEYRO1UYJBXNTS5WIGVWOAPAIMD543VKNEQD10DFDPW05
CLIENT_SECRET:BOYIADY50USFK3SDBSB52P3BAADB1YCET3ELHXIXWULXIQD2


In [22]:
radius=500
LIMIT=100

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            'BTCAEYRO1UYJBXNTS5WIGVWOAPAIMD543VKNEQD10DFDPW05', 
            'BOYIADY50USFK3SDBSB52P3BAADB1YCET3ELHXIXWULXIQD2', 
            'ymd', 
            43.653963, 
            -79.387207, 
            500, 
            100)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)